In [2]:
# from IPython.display import clear_output


# # !pip install tensorflow
# # !pip install  spacy
# # !pip install tqdm
# # !pip install plotly
# !pip install jupyter-black
# !pip install imblearn
# # !pip install transformers
# # !pip install sentencepiece
# !pip install catboost

# # clear_output()

In [3]:
# import transformers

## Data preprocessing

### Import libraries

In [4]:
import csv
import re
import numpy as np
from sklearn.metrics import accuracy_score, log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm

tqdm.pandas()

In [6]:
%load_ext jupyter_black

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [7]:
np.set_printoptions(precision=4)

In [29]:
def submit(y_pred_proba, name=""):
    # Write predictions to a file
    with open("../Submissions/" + name + "ML_Approach_ESM.csv", "w") as csvfile:
        writer = csv.writer(csvfile, delimiter=",")
        lst = list()
        for i in range(18):
            lst.append("class" + str(i))
        lst.insert(0, "name")
        writer.writerow(lst)
        for i, protein in enumerate(proteins_test):
            lst = y_pred_proba[i, :].tolist()
            lst.insert(0, protein)
            writer.writerow(lst)

In [9]:
### Important functions


def softmax(x):
    y = np.exp(x - np.max(x))
    f_x = y / np.sum(np.exp(x))
    return f_x


softmax_vect = np.vectorize(softmax)

# weighted multi-class log loss
from sklearn.metrics import log_loss


def weighted_mc_log_loss(y_true, y_pred, y_pred_proba):
    loss = log_loss(y_true, y_pred_proba, labels=np.unique(y_true))
    accuracy = round((y_true == y_pred).sum() / len(y_true) * 100, 2)
    return print(f"{loss = } and accuracy {accuracy = }")

### Import data

In [10]:
import time
import scipy.sparse as sp

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

path = "./data/"


def load_data():
    """
    Function that loads graphs
    """
    graph_indicator = np.loadtxt(path + "graph_indicator.txt", dtype=np.int64)
    _, graph_size = np.unique(graph_indicator, return_counts=True)

    edges = np.loadtxt(path + "edgelist.txt", dtype=np.int64, delimiter=",")
    edges_inv = np.vstack((edges[:, 1], edges[:, 0]))
    edges = np.vstack((edges, edges_inv.T))
    s = edges[:, 0] * graph_indicator.size + edges[:, 1]
    idx_sort = np.argsort(s)
    edges = edges[idx_sort, :]
    edges, idx_unique = np.unique(edges, axis=0, return_index=True)
    A = sp.csr_matrix(
        (np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
        shape=(graph_indicator.size, graph_indicator.size),
    )

    x = np.loadtxt(path + "node_attributes.txt", delimiter=",")
    edge_attr = np.loadtxt(path + "edge_attributes.txt", delimiter=",")
    edge_attr = np.vstack((edge_attr, edge_attr))
    edge_attr = edge_attr[idx_sort, :]
    edge_attr = edge_attr[idx_unique, :]

    adj = []
    features = []
    edge_features = []
    idx_n = 0
    idx_m = 0
    for i in range(graph_size.size):
        adj.append(A[idx_n : idx_n + graph_size[i], idx_n : idx_n + graph_size[i]])
        edge_features.append(edge_attr[idx_m : idx_m + adj[i].nnz, :])
        features.append(x[idx_n : idx_n + graph_size[i], :])
        idx_n += graph_size[i]
        idx_m += adj[i].nnz

    return adj, features, edge_features

In [11]:
def normalize_adjacency(A):
    """
    Function that normalizes an adjacency matrix
    """
    n = A.shape[0]
    A += sp.identity(n)
    degs = A.dot(np.ones(n))
    inv_degs = np.power(degs, -1)
    D = sp.diags(inv_degs)
    A_normalized = D.dot(A)

    return A_normalized


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """
    Function that converts a Scipy sparse matrix to a sparse Torch tensor
    """
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64)
    )
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [30]:
# Load graphs
# adj, features, edge_features = load_data()

# Normalize adjacency matrices
# adj = [normalize_adjacency(A) for A in adj]

# Split data into training and test sets
# adj_train = list()
# features_train = list()
# edge_features_train = list()
y_train = list()
# adj_test = list()
# features_test = list()
# edge_features_test = list()
proteins_test = list()

In [31]:
with open(path + "graph_labels.txt", "r") as f:
    for i, line in enumerate(f):
        t = line.split(",")
        if len(t[1][:-1]) == 0:
            pass
            proteins_test.append(t[0])
            # adj_test.append(adj[i])
            # features_test.append(features[i])
            # edge_features_test.append(edge_features[i])
        else:
            # adj_train.append(adj[i])
            # features_train.append(features[i])
            # edge_features_train.append(edge_features[i])
            y_train.append(int(t[1][:-1]))

In [23]:
import joblib as joblib

# features_train = joblib.load("new_features_3B_params.sav")
features_test = joblib.load("new_features_test_3B_params.sav")
n_input = features_train[0].shape[1]

In [16]:
features_train[0].shape, len(features_train)

((185, 2646), 4888)

In [24]:
# features_seq_train = [feature.sum(0) for feature in features_train]
features_seq_test = [feature.sum(0) for feature in features_test]

In [180]:
# features_seq_train_edges = []
# for i in range(len(features_train)):
#     feat_concatenated = np.concatenate(
#         [features_train[i].mean(0), 0.2 * edge_features[i].mean(0)], axis=0
#     )
#     features_seq_train_edges += [feat_concatenated]
# len(features_seq_train_edges), features_seq_train_edges[0].shape

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train_cat, y_test_cat = train_test_split(
    features_seq_train, y_train, test_size=0.06, random_state=42
)

In [19]:
from catboost import CatBoostClassifier

model1 = CatBoostClassifier(
    iterations=3000,
    task_type="GPU",
    devices="0:1",
    loss_function="MultiClass",
    use_best_model=True,  # loss_function="MultiClassOneVsAll"
)

# model1 = CatBoostClassifier(
#     iterations=1000,
#     depth=6,
#     learning_rate=0.1,
#     loss_function="MultiClass",
#     rsm=0.95,
#     border_count=64,
#     eval_metric="AUC",
#     l2_leaf_reg=3.5,
#     one_hot_max_size=30,
#     use_best_model=True,
#     verbose=False,
#     random_seed=502,
# )

model1.fit(X_train, y_train_cat, eval_set=(X_test, y_test_cat), verbose=False)

In [33]:
print(model1.get_best_iteration())

2735


In [21]:
print(model1.score(X_test, y_test_cat))
y_pred = model1.predict(X_test)
y_pred_proba_cat = model1.predict_proba(X_test)
weighted_mc_log_loss(y_test_cat, y_pred, y_pred_proba_cat)

0.7755102040816326


ValueError: The number of classes in labels is different from that in y_pred. Classes found in labels: [ 0  2  3  4  5  6  7  8 11 14 15 16]

In [25]:
y_pred_proba_cat = model1.predict_proba(features_seq_test)

In [28]:
y_pred_proba_cat[0].sum()

1.0

In [32]:
submit(y_pred_proba_cat, "3_B_")

In [160]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    max_depth=10, criterion="log_loss", n_estimators=1000, random_state=0
)
rf.fit(X_train, y_train_cat)

RandomForestClassifier(criterion='log_loss', max_depth=10, n_estimators=1000,
                       random_state=0)

In [161]:
print(rf.score(X_test, y_test_cat))
y_pred = rf.predict(X_test)
y_pred_proba_rf = rf.predict_proba(X_test)
weighted_mc_log_loss(y_test_cat, y_pred, y_pred_proba)

0.7096114519427403
loss = 1.1549422379770247 and accuracy accuracy = 70.96


In [148]:
y_pred_proba_cat_rf = (y_pred_proba_rf + y_pred_proba_cat) / 2

In [149]:
y_pred_proba_cat_rf = (y_pred_proba_rf + y_pred_proba_cat) / 2
log_loss(y_test_cat, y_pred_proba_cat_rf, labels=np.unique(y_test_cat))

0.921897223727007

## IGnore


In [27]:
import pandas as pd
import csv

ems_gcn = pd.read_csv("../Submissions/geometric_GCNcov_150_param.csv")
catboost = pd.read_csv("../Submissions/ML_Approach_ESM.csv")

In [ ]:
csv_total = pd.concat([catboost, ems_gcn], axis=1)
csv_total.shape

In [ ]:
np.concatenate([catboost.to_numpy()[:, 0],catboost.to_numpy()[:,1:]*0.55 + 0.45*ems_gcn.to_numpy()[:,1:]],axis=1)

In [ ]:
catboost.to_numpy()[:, 0]

In [66]:
df = catboost.to_numpy()[:, 1:] * 0.55 + 0.45 * ems_gcn.to_numpy()[:, 1:]

In [73]:
submit(df, "sum_cat_gcn")

# Weighted model

In [28]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv, GATConv, GINConv, global_add_pool, SAGEConv
from torch_geometric.nn import global_mean_pool

num_node_features = features_train[0].shape[1]
num_classes = 18

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


class GNN(torch.nn.Module):
    def __init__(
        self,
        model_name="GCN",
        hidden_channels=[64, 64, 32],
        num_heads=[1, 1, 1],
        dropout=0.02,
        n_classes=num_classes,
        input_dim=num_node_features,
    ):
        super(GNN, self).__init__()
        self.n_classes = n_classes
        self.dropout = dropout
        self.hidden_dim = hidden_channels[2]
        self.model_name = model_name

        if model_name == "GCN":
            self.layer1 = GCNConv(
                in_channels=input_dim, out_channels=hidden_channels[1]
            )
            self.layer2 = GCNConv(
                in_channels=hidden_channels[1], out_channels=hidden_channels[2]
            )
            # self.layer3 = GCNConv(
            #     in_channels=hidden_channels[1], out_channels=hidden_channels[2]
            # )

        elif model_name == "GAT":
            self.layer1 = GATConv(
                in_channels=input_dim,
                out_channels=hidden_channels[0],
                heads=num_heads[0],
                edge_dim=5,
            )
            self.layer2 = GATConv(
                hidden_channels[0] * num_heads[0],
                hidden_channels[1],
                heads=num_heads[1],
                edge_dim=5,
            )
            self.layer3 = GATConv(
                hidden_channels[1] * num_heads[1],
                hidden_channels[2],
                heads=1,
                edge_dim=5,
                concat=False,
            )

        elif model_name == "GraphSAGE":
            self.layer1 = SAGEConv(input_dim, hidden_channels[0], aggr="lstm")
            self.layer2 = SAGEConv(hidden_channels[0], hidden_channels[1], aggr="lstm")
            self.layer3 = SAGEConv(hidden_channels[1], hidden_channels[2], aggr="lstm")

        self.decoder = nn.Linear(hidden_channels[2], n_classes)

    def forward(self, x, edge_index, edge_attr, batch):
        if self.model_name == "GAT":
            x = F.dropout(x, p=self.dropout, training=self.training)
            x = self.layer1(x, edge_index, edge_attr)
            x = F.elu(x)

            x = F.dropout(x, p=self.dropout, training=self.training)
            x = self.layer2(x, edge_index, edge_attr)
            x = F.elu(x)

            x = F.dropout(x, p=self.dropout, training=self.training)
            x = self.layer3(x, edge_index, edge_attr)
        else:
            x = F.dropout(x, p=self.dropout, training=self.training)
            x = self.layer1(x, edge_index)
            x = F.elu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)

            x = self.layer2(x, edge_index)

            # x = F.relu(x)
            # x = F.dropout(x, p=self.dropout, training=self.training)
            # x = self.layer3(x, edge_index)
        x = F.elu(x)
        x = global_mean_pool(x, batch)
        # x = global_add_pool(x, batch)
        # x = F.dropout(x, p=self.dropout, training=self.training)

        x = self.decoder(x)
        return x
        # return F.log_softmax(x, dim=-1)


model = GNN().to(device)
print(model)

GNN(
  (layer1): GCNConv(726, 64)
  (layer2): GCNConv(64, 32)
  (decoder): Linear(in_features=32, out_features=18, bias=True)
)


In [29]:
model.load_state_dict(torch.load("saved_model_150m_params_GAT.pth"))
model.eval()

GNN(
  (layer1): GCNConv(726, 64)
  (layer2): GCNConv(64, 32)
  (decoder): Linear(in_features=32, out_features=18, bias=True)
)

In [30]:
softmax = nn.Softmax(dim=1)


def predict(loader):
    model.eval()

    correct = 0
    pred = []
    for data in loader:  # Iterate in batches over the training/test dataset.
        data.to(device)
        out = model(data.x, data.edge_index, _, data.batch)
        out = softmax(out)
        pred.append(out.to("cpu").numpy())
        # pred.append(out.argmax(dim=1))  # Use the class with highest probability.

    return np.concatenate(pred, axis=0)  # Derive ratio of correct predictions. , proba

In [31]:
from torch_geometric.data import Data


def get_Data(adj_train, features_train, edge_features_train, y_train):
    data = []
    for i in range(len(features_train)):
        adj_t = torch.tensor(adj_train[i].todense())
        edge_index = adj_t.nonzero().t().contiguous()
        x = torch.tensor(features_train[i]).float()
        edge_attr = torch.tensor(edge_features_train[i]).float()

        data.append(Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y_train[i]))
    return data


data_obj = get_Data(adj_train, features_train, edge_features_train, y_train)

In [32]:
def get_Data_pred(adj_train, features_train, edge_features_train):
    data = []
    for i in range(len(features_train)):
        adj_t = torch.tensor(adj_train[i].todense())
        edge_index = adj_t.nonzero().t().contiguous()
        x = torch.tensor(features_train[i]).float()
        edge_attr = torch.tensor(edge_features_train[i]).float()

        data.append(Data(x=x, edge_index=edge_index, edge_attr=edge_attr))
    return data


data_obj_pred = get_Data_pred(adj_test, features_test, edge_features_test)

In [33]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(data_obj, batch_size=32, shuffle=False)
pred_loader = DataLoader(data_obj_pred, batch_size=32, shuffle=False)

In [34]:
with torch.no_grad():
    predictions_proba_gcn = predict(train_loader)

In [101]:
with torch.no_grad():
    predictions_proba_gcn_sub = predict(pred_loader)

In [100]:
# predictions_proba_catboost = model1.predict_proba(features_seq_train)  # target_train
predictions_proba_catboost_sub = model1.predict_proba(features_seq_test)  # target_train

In [38]:
predictions_proba_gcn.shape == predictions_proba_catboost.shape

True

In [39]:
dataset_prob = np.concatenate(
    [predictions_proba_catboost, predictions_proba_gcn], axis=1
)

In [102]:
dataset_prob_sub = np.concatenate(
    [predictions_proba_catboost_sub, predictions_proba_gcn_sub], axis=1
)

In [40]:
dataset_prob.shape

(4888, 36)

In [41]:
X_train_ada, X_test_ada, y_train_ada, y_test_ada = train_test_split(
    dataset_prob, y_train, test_size=0.10, random_state=42
)

In [97]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(penalty="elasticnet", l1_ratio=0.0, C=0.5, solver="saga")
clf.fit(X_train_ada, y_train_ada)

LogisticRegression(C=0.5, l1_ratio=0.0, penalty='elasticnet', solver='saga')

In [103]:
pred_ada = clf.predict(X_test_ada)
pred_proba_ada = clf.predict_proba(dataset_prob_sub)

# clf.score(X_test_ada, y_test_ada)

In [99]:
log_loss(y_test_ada, pred_proba_ada, labels=np.unique(y_test_ada))

0.9415871087485389

In [104]:
submit(pred_proba_ada, "tkharbi9a")

In [57]:
pred = (predictions_proba_catboost + predictions_proba_gcn) / 2